# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [179]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [180]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'

In [181]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
# database = ''

In [182]:
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [183]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

In [184]:
pd.read_sql_query("SHOW TABLES", conn)

,Tables_in_hds5210
0,data
1,ecdmdphd_data
2,ed_visits
3,hospital_services
4,hospitals
5,lillyeversman_data
6,paulboal_data
7,payment_data
8,per_data
9,population


In [185]:
tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

data
ecdmdphd_data
ed_visits
hospital_services
hospitals
lillyeversman_data
paulboal_data
payment_data
per_data
population
sravani9544_data
svelicheti_data
system_affiliations
total_per_population


In [186]:
tables['population']

,index,Zip,Population
0,0,90011,110750
1,1,90650,104524
2,2,91331,103683
3,3,90201,100512
4,4,90044,99980
...,...,...,...
1645,1645,95699,13
1646,1646,96119,11
1647,1647,92304,7
1648,1648,93530,6


## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [187]:
#import csv file using pandas pd 

import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/home/jupyter-msimkins7/hds5210-2022/Electronic_Health_Record_EHR_Incentive_Program_Payments_to_Eligible_Hospitals.csv')

In [188]:
MYTABLE = pd.read_sql_query('SELECT * FROM data', con=conn)

In [193]:
MYTABLE.columns

Index(['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'],
      dtype='object')

In [194]:
MYTABLE.shape

(323, 22)

In [195]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [196]:
population = tables['population']

In [197]:
answer = pd.merge(MYTABLE,
                  population,
                  left_on = 'Business_ZIP_Code',
                  right_on = 'Zip',
                  how = 'inner')

In [198]:
answer

,index_x,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,...,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year,index_y,Zip,Population
0,0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,...,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016,736,95501,23948
1,1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,...,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016,543,94545,32552
2,194,-122.096165,37.630272,195,KAISER FOUNDATION HOSPITALS,1801960513,50512,27400 Hesperian Blvd.,Hayward,Alameda,...,415091.50,MU,41509.15,37.630272,-122.096165,2018,2020,543,94545,32552
3,2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,...,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016,77,94558,66854
4,3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,...,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019,1067,95642,6547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,312,-122.907916,39.105474,313,Sutter Bay Hospitals,1952634008,51329,5176 HILL ROAD EAST,LAKEPORT,Lake,...,1142752.33,MU,114275.23,39.105474,-122.907916,2016,2019,972,95453,11209
313,313,-122.267252,37.837338,314,Children's Hospital & Research Center at Oakland,1003961251,53301,747 52nd St,Oakland,Alameda,...,6760775.32,MU,676077.53,37.837338,-122.267252,2016,2019,741,94609,23729
314,316,-117.833859,33.782715,317,HEALTHBRIDGE CHILDREN'S HOSPITAL-ORANGE CA LP,1225015985,53308,393 S Tustin St,Orange,Orange,...,1543558.29,MU,578834.36,33.782715,-117.833859,2015,2017,888,92866,15428
315,318,-122.173214,37.436328,319,LUCILE SALTER PACKARD CHILDREN'S HOSPITAL AT S...,1467442749,53305,"725 Welch Rd, mail code 5500",Palo Alto,Santa Clara,...,5630849.63,MU,563084.96,37.436328,-122.173214,2017,2019,1133,94304,4391


In [125]:
answer['NPI']

0      1609858950
1      1942298153
2      1801960513
3      1235218785
4      1447494323
          ...    
312    1952634008
313    1003961251
314    1225015985
315    1467442749
316    1508908591
Name: NPI, Length: 317, dtype: int64

In [200]:
max_pop = answer['Population'].max()

In [201]:
max_pop

104524

In [202]:
filter = answer['Population'] ==104524

In [203]:
max_pop_only = answer[filter]

In [204]:
max_pop_only

,index_x,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,...,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year,index_y,Zip,Population
280,275,-118.09915,33.912613,276,"CPH HOSPITAL MANAGEMENT, LLC",1194016923,50771,13100 Studebaker Rd,Norwalk,Los Angeles,...,2184880.1,MU,218488.01,33.912613,-118.09915,2018,2020,1,90650,104524


In [205]:
answer = []

In [206]:
answer.append(max_pop_only[['NPI']])

In [211]:
answer

[            NPI
 280  1194016923]

In [212]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

## Now using Pandas instead of SQL

In [256]:
import numpy as np
test = data.pivot_table(
    values = 'total_payments',
    index= ['Business_ZIP_Code', 'Medicaid_EP_Hospital_Type'],
    #columns = 'Medicaid_EP_Hospital_Type',
    aggfunc = np.ma.mean,
    fill_value = 0)

In [262]:
test.reset_index()

,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments
0,90004,Acute Care Hospitals,1093021.73
1,90012,Acute Care Hospitals,3614739.93
2,90015,Acute Care Hospitals,10045484.04
3,90017,Acute Care Hospitals,3899845.21
4,90020,Children's Hospitals,427640.22
...,...,...,...
262,96104,Acute Care Hospitals,184615.39
263,96122,Acute Care Hospitals,531907.08
264,96130,Acute Care Hospitals,1371962.88
265,96150,Acute Care Hospitals,786769.13


In [264]:
#merge data with population data
new_test = pd.merge(test.reset_index(),
                  population,
                  left_on = 'Business_ZIP_Code',
                  right_on = 'Zip',
                  how = 'inner')

In [265]:
new_test

,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments,index,Zip,Population
0,90004,Acute Care Hospitals,1093021.73,115,90004,61105
1,90012,Acute Care Hospitals,3614739.93,433,90012,37268
2,90015,Acute Care Hospitals,10045484.04,738,90015,23900
3,90017,Acute Care Hospitals,3899845.21,648,90017,27832
4,90020,Children's Hospitals,427640.22,390,90020,39366
...,...,...,...,...,...,...
256,96104,Acute Care Hospitals,184615.39,1388,96104,920
257,96122,Acute Care Hospitals,531907.08,1169,96122,3594
258,96130,Acute Care Hospitals,1371962.88,810,96130,19859
259,96150,Acute Care Hospitals,786769.13,595,96150,30369


In [266]:
#calculate total payment per person based on zip population
new_test['total_per_person'] = (new_test["total_payments"])/new_test['Population']

In [267]:
new_test

,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments,index,Zip,Population,total_per_person
0,90004,Acute Care Hospitals,1093021.73,115,90004,61105,17.887599
1,90012,Acute Care Hospitals,3614739.93,433,90012,37268,96.993129
2,90015,Acute Care Hospitals,10045484.04,738,90015,23900,420.313140
3,90017,Acute Care Hospitals,3899845.21,648,90017,27832,140.120912
4,90020,Children's Hospitals,427640.22,390,90020,39366,10.863187
...,...,...,...,...,...,...,...
256,96104,Acute Care Hospitals,184615.39,1388,96104,920,200.668902
257,96122,Acute Care Hospitals,531907.08,1169,96122,3594,147.998631
258,96130,Acute Care Hospitals,1371962.88,810,96130,19859,69.085195
259,96150,Acute Care Hospitals,786769.13,595,96150,30369,25.906982


In [269]:
import numpy as np
answer = new_test.pivot_table(
    values = 'total_per_person',
    index= 'Medicaid_EP_Hospital_Type',
    aggfunc = np.ma.mean,
    fill_value = 0)

In [270]:
answer

,total_per_person
Medicaid_EP_Hospital_Type,
Acute Care Hospitals,85.444853
Children's Hospitals,315.394944


In [271]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


KeyError: 'Avg_Pay_per_Capita'



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')